In [1]:
# CIS 545

# Homework 5: Spam Classification in SciKit-Learn and TensorFlow

This assignment uses data from https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection

Data processing was inspired by https://www.kaggle.com/overflow012/d/uciml/sms-spam-collection-dataset/text-preprocessing-classification

The code below gives you a helper function and reads in the data.

In [2]:
import pandas as pd

# This function returns the k most frequently appearing keywords in the dataframe
def top_k(data_df, vec, k):
    X = vec.fit_transform(data_df['sms'].values)
    labels = vec.get_feature_names()    
    return pd.DataFrame(columns = labels, data = X.toarray()).sum().sort_values(ascending = False)[:k]

sms_df = pd.read_csv('spam.csv', encoding='latin-1')
sms_df.columns = ['class', 'sms', 'a', 'b', 'c']

## Step 1.1 Data Wrangling and Inspection

Clean up `sms_df`.  Delete 'a', 'b', 'c', lowercase the sms text, add a column 'length'.

In [3]:
# TODO: Data wrangling / cleaning
# YOUR CODE HERE
try:
    sms_df = sms_df.drop(['a','b','c'], axis = 1)
    sms_df['sms'] = sms_df['sms'].str.lower()
    
except:
    raise NotImplementedError()

In [4]:
#create a function to count length of the text
def textCount(text):
    return (len(text))

#apply the function and create new column called 'length'
sms_df['length'] = sms_df['sms'].apply(lambda x: textCount(x))


After you’ve done this, step through these cells to see how the input data divides between ‘spam’ texts and ‘ham’ (non-spam) texts.  You should note that the spam has certain terms, e.g., ‘winner’, that don’t appear as frequently in “ham.”

In [5]:
display(sms_df)

class                                                sms  length
0      ham  go until jurong point, crazy.. available only ...     111
1      ham                      ok lar... joking wif u oni...      29
2     spam  free entry in 2 a wkly comp to win fa cup fina...     155
3      ham  u dun say so early hor... u c already then say...      49
4      ham  nah i don't think he goes to usf, he lives aro...      61
5     spam  freemsg hey there darling it's been 3 week's n...     148
6      ham  even my brother is not like to speak with me. ...      77
7      ham  as per your request 'melle melle (oru minnamin...     160
8     spam  winner!! as a valued network customer you have...     158
9     spam  had your mobile 11 months or more? u r entitle...     154
10     ham  i'm gonna be home soon and i don't want to tal...     109
11    spam  six chances to win cash! from 100 to 20,000 po...     136
12    spam  urgent! you have won a 1 week free membership ...     156
13     ham  i've been searching for the right words to tha...     196
14     ham                i have a date on sunday with will!!      35
15    spam  xxxmobilemovieclub: to use your credit, click ...     149
16     ham                         oh k...i'm watching here:)      26
17     ham  eh u remember how 2 spell his name... yes i di...      81
18     ham  fine if thatåõs the way u feel. thatåõs the wa...      58
19    spam  england v macedonia - dont miss the goals/team...     156
20     ham          is that seriously how you spell his name?      41
21     ham  iû÷m going to try for 2 months ha ha only joking      49
22     ham  so ì_ pay first lar... then when is da stock c...      53
23     ham  aft i finish my lunch then i go str down lor. ...      88
24     ham  ffffffffff. alright no way i can meet up with ...      57
25     ham  just forced myself to eat a slice. i'm really ...     144
26     ham                     lol your always so convincing.      30
27     ham  did you catch the bus ? are you frying an egg ...     134
28     ham  i'm back &amp; we're packing the car now, i'll...      75
29     ham  ahhh. work. i vaguely remember that! what does...      64
...    ...                                                ...     ...
5542   ham           armand says get your ass over to epsilon      40
5543   ham             u still havent got urself a jacket ah?      38
5544   ham  i'm taking derek &amp; taylor to walmart, if i...     154
5545   ham      hi its in durban are you still on this number      45
5546   ham         ic. there are a lotta childporn cars then.      42
5547  spam  had your contract mobile 11 mnths? latest moto...     160
5548   ham                 no, i was trying it all weekend ;v      34
5549   ham  you know, wot people wear. t shirts, jumpers, ...      87
5550   ham        cool, what time you think you can get here?      43
5551   ham  wen did you get so spiritual and deep. that's ...      51
5552   ham  have a safe trip to nigeria. wish you happines...      91
5553   ham                        hahaha..use your brain dear      27
5554   ham  well keep in mind i've only got enough gas for...      98
5555   ham  yeh. indians was nice. tho it did kane me off ...     153
5556   ham  yes i have. so that's why u texted. pshew...mi...      63
5557   ham  no. i meant the calculation is the same. that ...     273
5558   ham                             sorry, i'll call later      22
5559   ham  if you aren't here in the next  &lt;#&gt;  hou...      66
5560   ham                  anything lor. juz both of us lor.      33
5561   ham  get me out of this dump heap. my mom decided t...      70
5562   ham  ok lor... sony ericsson salesman... i ask shuh...      96
5563   ham                                ard 6 like dat lor.      19
5564   ham  why don't you wait 'til at least wednesday to ...      67
5565   ham                                       huh y lei...      12
5566  spam  reminder from o2: to get 2.50 pounds free call...     147
5567  spam  thi

In [6]:
display(sms_df.groupby('class').describe())

length                                                         
        count        mean        std   min    25%    50%    75%    max
class                                                                 
ham    4825.0   71.023627  58.016023   2.0   33.0   52.0   92.0  910.0
spam    747.0  138.866131  29.183082  13.0  132.5  149.0  157.0  224.0

## Step 1.2. Vectorizing the Text

`scikit-learn` has a handy `CountVectorizer` that builds a (sparse) matrix of word counts, drops stop words and even does stemming.  See:

http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

The code below builds document vectors automatically.

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer(decode_error = 'ignore', stop_words = 'english')
X = vec.fit_transform(sms_df['sms'].values)

### Step 1.2.1 What are the frequent words? What are frequent patterns?

Run the next couple of cells to see the top-30 words in spam and in ham.  If you look, you should see numbers and parts of URLs (“www”, “com”) in the spam.  Perhaps we need to get rid of these altogether.  However, the URLs and numbers are highly varied, so we would like to map all URLs to one feature, and all numbers to one feature. 

In the cell below store the index of "www" as `www` and the index of "com" as `com`.

In [8]:
www = 0
com = 0
# YOUR CODE HERE
try:
    www = vec.vocabulary_['www']
    com = vec.vocabulary_['com']
except:
    raise NotImplementedError()

In [9]:
top_spam = top_k(sms_df[sms_df['class'] == 'spam'], vec, 30)
display(top_spam)

free          224
txt           163
ur            144
mobile        127
text          125
stop          121
claim         113
reply         104
www            98
prize          93
just           78
cash           76
won            76
uk             74
150p           71
send           70
new            69
nokia          67
win            64
urgent         63
tone           60
week           60
50             57
contact        56
service        56
msg            54
com            54
18             51
16             51
guaranteed     50
dtype: int64

In [10]:
top_ham = top_k(sms_df[sms_df['class'] == 'ham'], vec, 30)
display(top_ham)

gt       318
lt       316
just     293
ok       287
ll       265
ur       241
know     236
good     233
got      232
like     232
come     227
day      209
time     201
love     199
going    169
home     165
want     164
lor      162
need     158
sorry    157
don      151
da       150
today    139
later    135
dont     132
did      129
send     129
think    128
pls      123
hi       122
dtype: int64

### Step 1.2.2 Regularize URLs and Numbers

Let’s replace all URL patterns, and all numbers, with a single text token (“_url_” and “_number_”).  To do this, simply pass the appropriate column of your DataFrame to `regularize_urls` and `regularize_numbers`. Replace the SMS text with the results of regularizing.

In [11]:
# TODO: Regularize/tokenize URLs and numbers
from regularize import regularize_urls, regularize_numbers
# YOUR CODE HERE
try:
    sms_df['sms'] = regularize_urls(sms_df['sms'])
    sms_df['sms'] = regularize_numbers(sms_df['sms'])
except:
    raise NotImplementedError()

## Step 1.2.2 Results

Re-run the CountVectorizer, re-create vector `X`, and re-compute the top 30 spam terms.  Store the top 30 spam terms in `top_spam`.

In [12]:
# TODO: Top-30 spam terms
# YOUR CODE HERE
try:
    X = vec.fit_transform(sms_df['sms'].values)
    top_spam = top_k(sms_df[sms_df['class'] == 'spam'],vec, 30)
except:
    raise NotImplementedError()

In [13]:
display(top_spam)

_num_         3289
free           228
txt            165
ur             144
_url_          141
mobile         129
stop           126
text           125
claim          113
reply          104
prize           92
just            78
won             76
cash            76
nokia           71
send            70
win             70
new             69
urgent          63
week            60
tone            59
box             57
msg             56
service         56
contact         56
guaranteed      50
ppm             49
customer        49
mins            47
phone           46
dtype: int64

In [14]:
if "_num_" not in top_spam:
    raise ValueError

In [15]:
if "_url_" not in top_spam:
    raise ValueError

## Step 1.3 Creating Features

Currently we have a very large number of features, namely all of the words that aren’t stop words.  Let’s do dimensionality reduction, by only looking for the words that frequently occur in either spam or ham.  Recall that we just recomputed the top-30 spam words.

Compute the top-30 ham words, then create a list of “vocabulary” words from the combination of the top spam + ham words.  Create a new feature set called `relevant_vec` using `CountVectorizer` with just the top spam + ham words. 

Hint: See http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html.

In [16]:
# TODO: Vector of 'important' words
# YOUR CODE HERE
try:
    top_spam = top_k(sms_df[sms_df['class'] == 'spam'], vec, 30)
    top_ham = top_k(sms_df[sms_df['class'] == 'ham'], vec, 30)
    vocab_lst = list(set(top_ham.keys()).union(set(top_spam.keys())))
    relevant_vec = CountVectorizer(decode_error = 'ignore',vocabulary = vocab_lst)
    relevant_vec
    
except:
    raise NotImplementedError()

In [17]:
display(relevant_vec)

CountVectorizer(analyzer='word', binary=False, decode_error='ignore',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None,
        vocabulary=['lt', '_num_', 'don', 'send', 'gt', 'think', 'service', 'mobile', 'prize', 'got', 'like', 'mins', 'text', 'free', 'just', '_url_', 'ok', 'good', 'ppm', 'later', 'tone', 'dont', 'time', 'win', 'lor', 'claim', 'msg', 'stop', 'reply', 'home', 'new', 'need', 'phone', 'tell', 'box', 'cash', 'day', 'know', 'won', 'll', 'love', 'today', 'contact', 'come', 'ur', 'week', 'txt', 'want', 'guaranteed', 'sorry', 'customer', 'urgent', 'going', 'did', 'nokia', 'da'])

This cell adds an sms length feature (normalized by the maximum length) and creates training and test sets for you.

In [18]:
import sklearn.model_selection as ms
from sklearn.feature_extraction.text import TfidfTransformer
import numpy as np

# X is the feature array, based off relevant words
X = relevant_vec.fit_transform(sms_df['sms'].values).toarray()

# Compute the length of each sms message, normalized
# by max length
Xlen = np.zeros((X.shape[0],1))
inx = 0
for v in sms_df['sms'].values:
        Xlen[inx,0] = len(v)
        inx += 1
Xlen = Xlen / max(Xlen)
# Add the length as another feature
X = np.hstack((X, Xlen))

#y = sms_df['class'].values
y = np.array((sms_df['class'] == 'spam').astype(int))

# Now we split...
X_train, X_test, y_train, y_test = ms.train_test_split(X, y, test_size=0.2, random_state = 42)

X_train

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.09110867],
       [0.        , 3.        , 0.        , ..., 0.        , 0.        ,
        0.16684962],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.05049396],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.04939627],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.02854007],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.03841932]])

## Step 1.4 Classifier Evaluation

Write a function `spam_classify(model)` that takes a constructed model as input, `fit`s the model on the training set, `predict`s on the test set, and returns the `score` on the test outputs. The monospaced words are hints for which functions you should use. Note that you may need to run the cell below again as your write the constructor calls so that you do not get duplicates in your results table.

In [19]:
# TODO: Write your spam_classify function here.
# YOUR CODE HERE
try:
    def spam_classify(model):
        model.fit(X_train, y_train)
        test_class = model.predict(X_test)
        score = np.sum(test_class == y_test)/len(X_test)
        return score
except:
    raise NotImplementedError()

# Results, as a list of dictionaries
classifier_results = []

### Step 1.4.1 Decision Trees

Construct 6 Decision Trees **with random state 42**. Use maximum depths of 1, 2, 3, 4, 5, and the default. Store them in variables `dt1` to `dt6`.

In [20]:
from sklearn.tree import DecisionTreeClassifier
import sklearn.model_selection as ms

# TODO: Construct your 5 decision trees here.
# YOUR CODE HERE
try:
    dt1 = DecisionTreeClassifier(max_depth = 1, random_state = 42)
    dt2 = DecisionTreeClassifier(max_depth = 2, random_state = 42)
    dt3 = DecisionTreeClassifier(max_depth = 3, random_state = 42)
    dt4 = DecisionTreeClassifier(max_depth = 4, random_state = 42)
    dt5 = DecisionTreeClassifier(max_depth = 5, random_state = 42)
    dt6 = DecisionTreeClassifier(max_depth = 6, random_state = 42)
    
except:
    raise NotImplementedError()

classifier_results.append({'Classifier': 'DecTree', 'Param': 'Depth=1', 'Score': spam_classify(dt1)})
classifier_results.append({'Classifier': 'DecTree', 'Param': 'Depth=2', 'Score': spam_classify(dt2)})
classifier_results.append({'Classifier': 'DecTree', 'Param': 'Depth=3', 'Score': spam_classify(dt3)})
classifier_results.append({'Classifier': 'DecTree', 'Param': 'Depth=4', 'Score': spam_classify(dt4)})
classifier_results.append({'Classifier': 'DecTree', 'Param': 'Depth=5', 'Score': spam_classify(dt5)})
classifier_results.append({'Classifier': 'DecTree', 'Param': '', 'Score': spam_classify(dt6)})

### Step 1.4.2 Logistic Regression

Construct 2 `liblinear` Logistic Regression classifiers **with random state 42**. Use `l1` and `l2` regularization penalties. Store them in variables `lr1` and `lr2`.

In [21]:
from sklearn.linear_model import LogisticRegression

# TODO: Construct your 2 logistic regression classifiers here.
# YOUR CODE HERE
try:
    lr1 = LogisticRegression(random_state = 42, penalty = 'l1')
    lr2 = LogisticRegression(random_state = 42, penalty = 'l2')
except:
    raise NotImplementedError()

classifier_results.append({'Classifier': 'LogReg',  'Param': 'Reg=l1',  'Score': spam_classify(lr1)})
classifier_results.append({'Classifier': 'LogReg',  'Param': 'Reg=l2',  'Score': spam_classify(lr2)})

/usr/local/lib/python3.5/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


### Step 1.4.3 Support Vector Machines

Construct 1 Support Vector Machines classifier with the kernel coefficient set by `gamma="auto"` and **with random state 42**. Ensure the model computes probabilities for the 2 classes by setting the probability flag to True. Store the model in the variable `svm`.

In [22]:
from sklearn.svm import SVC

# TODO: Construct your 1 support vector machines classifier here.
# YOUR CODE HERE
try:
    svm = SVC(random_state = 42, probability = True, gamma = 'auto')
    
    
except:
    raise NotImplementedError()

classifier_results.append({'Classifier': 'SVM', 'Param': '', 'Score': spam_classify(svm)})

In [23]:
display(pd.DataFrame(classifier_results))

Classifier    Param     Score
0    DecTree  Depth=1  0.939910
1    DecTree  Depth=2  0.939910
2    DecTree  Depth=3  0.947085
3    DecTree  Depth=4  0.953363
4    DecTree  Depth=5  0.960538
5    DecTree           0.964126
6     LogReg   Reg=l1  0.971300
7     LogReg   Reg=l2  0.970404
8        SVM           0.971300

In [24]:
# Don't delete this cell.

In [25]:
# Don't delete this cell.

In [26]:
# Don't delete this cell.

## Step 2.0 Ensembles

We are going to use your `spam_classify` function again. No changes needed. Note that you may need to run the cell below again as your write the constructor calls so that you do not get duplicates in your results table.

In [27]:
# Results, as a list of dictionaries
classifier_results = []

## Step 2.1 Random Forest

Construct 1 random forest classifier with 31 estimators and **random state 314**. Store it in the variable `rfor`.

In [28]:
from sklearn.ensemble import RandomForestClassifier

# TODO: Construct your random forest classifier here.
# YOUR CODE HERE
try:
    rfor = RandomForestClassifier(n_estimators = 31, random_state = 114)
except:
    raise NotImplementedError()

classifier_results.append({'Classifier': 'RFo', 'Param': 'Count=31', 'Score': spam_classify(rfor)})

## Step 2.2 Bagging

Construct 4 bagging classifiers with 31 estimators and **random state 314**. The base classifiers should be `dt6`, `lr1`, `lr2`, and `svm`. Store them in variables `bag1` to `bag4`.

In [29]:
from sklearn.ensemble import BaggingClassifier

# TODO: Construct your bagging classifier here.
# YOUR CODE HERE
try:
    bag1 = BaggingClassifier(n_estimators = 31,base_estimator = dt6, random_state = 314)
    bag2 = BaggingClassifier(n_estimators = 31,base_estimator = lr1, random_state = 314)
    bag3 = BaggingClassifier(n_estimators = 31,base_estimator = lr2, random_state = 314)
    bag4 = BaggingClassifier(n_estimators = 31,base_estimator = svm, random_state = 314)
except:
    raise NotImplementedError()

classifier_results.append({'Classifier': 'Bag', 'Param': 'Base=dt6', 'Score': spam_classify(bag1)})
classifier_results.append({'Classifier': 'Bag', 'Param': 'Base=lr1', 'Score': spam_classify(bag2)})
classifier_results.append({'Classifier': 'Bag', 'Param': 'Base=lr2', 'Score': spam_classify(bag3)})
classifier_results.append({'Classifier': 'Bag', 'Param': 'Base=svm', 'Score': spam_classify(bag4)})

/usr/local/lib/python3.5/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  Futur

/usr/local/lib/python3.5/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  Futur

## Step 2.3 AdaBoost

Construct 4 AdaBoost classifiers with 31 estimators and **random state 314**. The base classifiers should be `dt6`, `lr1`, `lr2`, and `svm`. Store them in variables `ada1` to `ada4`. Training these models could take a long time.

In [30]:
from sklearn.ensemble import AdaBoostClassifier

# TODO: Construct your AdaBoost Classifier here.
# YOUR CODE HERE
try:
    ada1 = AdaBoostClassifier(n_estimators = 31, base_estimator = dt6, random_state = 314)
    ada2 = AdaBoostClassifier(n_estimators = 31, base_estimator = lr1, random_state = 314)
    ada3 = AdaBoostClassifier(n_estimators = 31, base_estimator = lr2, random_state = 314)
    ada4 = AdaBoostClassifier(n_estimators = 31, base_estimator = svm, random_state = 314)
except:
    raise NotImplementedError()

classifier_results.append({'Classifier': 'Ada', 'Param': 'Base=dt6', 'Score': spam_classify(ada1)})
classifier_results.append({'Classifier': 'Ada', 'Param': 'Base=lr1', 'Score': spam_classify(ada2)})
classifier_results.append({'Classifier': 'Ada', 'Param': 'Base=lr2', 'Score': spam_classify(ada3)})
classifier_results.append({'Classifier': 'Ada', 'Param': 'Base=svm', 'Score': spam_classify(ada4)})

/usr/local/lib/python3.5/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  Futur

/usr/local/lib/python3.5/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  Futur

In [31]:
display(pd.DataFrame(classifier_results))

Classifier     Param     Score
0        RFo  Count=31  0.981166
1        Bag  Base=dt6  0.971300
2        Bag  Base=lr1  0.973991
3        Bag  Base=lr2  0.971300
4        Bag  Base=svm  0.973094
5        Ada  Base=dt6  0.975785
6        Ada  Base=lr1  0.865471
7        Ada  Base=lr2  0.950673
8        Ada  Base=svm  0.919283

In [32]:
if min(pd.DataFrame(classifier_results)["Score"][0:5]) < 0.95:
    raise ValueError("Something went wrong.")

In [33]:
# Don't delete this cell.

## Step 3.0 Neural Networks

Let’s continue building upon our spam classifier, this time using neural networks -- both Perceptrons and feed-forward networks. We are going to use your `spam_classify` function again. No changes needed. Note that you may need to run the cell below again as your write the constructor calls so that you do not get duplicates in your results table.

In [34]:
# Results, as a list of dictionaries
classifier_results = []

## Step 3.1 Perceptron

Construct 1 perception classifier with a maximum number of iterations of 1000, a tolerance of 1e-3, and **random state 42**. Store it in the variable `perc`.

In [35]:
from sklearn.linear_model import Perceptron

# TODO: Construct your perception classifier here.
# YOUR CODE HERE
try:
    perc = Perceptron(max_iter = 1000,tol = 1e-3,random_state = 42)
except:
    raise NotImplementedError()

classifier_results.append({'Classifier': 'Perceptron', 'Param': '', 'Score': spam_classify(perc)})

## Step 3.2 Multi-layer Perceptrons

### Step 3.2.1 Small Network

Construct 1 MLP classifier with 3 hidden nodes in 1 layer and **random state 42**. Store it in the variable `mlp1`.

In [36]:
from sklearn.neural_network import MLPClassifier

# TODO: Construct your small MLP classifier here.
# YOUR CODE HERE
try:
    mlp1 = MLPClassifier(hidden_layer_sizes = (3,), random_state = 42)
except:
    raise NotImplementedError()

classifier_results.append({'Classifier': 'NN', 'Param': 'Hidden=(3)', 'Score': spam_classify(mlp1)})

### Step 3.2.2 Medium Network

Construct 1 MLP classifier with 10 hidden nodes in 1 layer and **random state 42**. Store it in the variable `mlp2`.

In [37]:
# TODO: Construct your medium MLP classifier here.
# YOUR CODE HERE
try:
    mlp2 = MLPClassifier(hidden_layer_sizes = (10,), random_state = 42)
except:
    raise NotImplementedError()

classifier_results.append({'Classifier': 'NN', 'Param': 'Hidden=(10)', 'Score': spam_classify(mlp2)})

### Step 3.2.3 Large Network

Construct 1 MLP classifier with 10 hidden nodes in each of 3 layers and **random state 1**. Store it in the variable `mlp3`.

In [38]:
# TODO: Construct your large MLP classifier here.
# YOUR CODE HERE
try:
    mlp3 = MLPClassifier(hidden_layer_sizes= (10,10,10), random_state = 1 )
except:
    raise NotImplementedError()

classifier_results.append({'Classifier': 'NN', 'Param': 'Hidden=(10,10,10)', 'Score': spam_classify(mlp3)})

/usr/local/lib/python3.5/dist-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [39]:
display(pd.DataFrame(classifier_results))

Classifier              Param     Score
0  Perceptron                     0.973094
1          NN         Hidden=(3)  0.976682
2          NN        Hidden=(10)  0.976682
3          NN  Hidden=(10,10,10)  0.973094

In [40]:
# Don't delete this cell.

In [41]:
# Don't delete this cell.

In [42]:
# Don't delete this cell.

# Tensorflow

## Step 4.1. Loading Data

The first cell imports TensorFlow. For this part of the homework, you will use `X_train` and `y_train` created in Step 1.3.

Define TensorFlow **columns** (features) for each of the words in the vocabulary from Step 1.3.  Also add an additional column for the length. Store these columns in a list.

In [46]:
import tensorflow as tf

# TODO: Define TensorFlow columns
try:
    label_col = []
    vocab_lst.append('length')
    for i in vocab_lst:
        label_col.append(tf.feature_column.numeric_column(i))
    label_col
# YOUR CODE HERE
except:
    raise NotImplementedError()

## Step 4.2. Setting up Features

Create a function `input_fn` that takes parameters `x` (2D numpy array of features) and `y` (1D numpy array of labels).  This should create a tensor for each **column** of the 2D array `x`. You can think of this as creating a tensor for each feature. This function should return a tuple of the dictionary of the tensors created from the columns and a tensor created from the second input `y`.  

Create a function `test_input_fn` that takes no arguments, but returns the output of passing in the test set and labels to `input_fn`. Create a similar function `train_input_fn` that does the same thing except passes in the training set and labels.

In [56]:
# TODO: Create 3 functions: input_fn(x,y), train_input_fn(), test_input_fn()
# YOUR CODE HERE
try:
    def input_fn(x,y):
        features = {k:(tf.constant(x[:,i]) if k!= 'length' else tf.constant(x[:,-1])) for i,k in enumerate(vocab_lst)}
        label = tf.constant(y)
        return features,label
                       
    def train_input_fn():
        return input_fn(X_train, y_train)
    
    def test_input_fn():
        return input_fn(X_test, y_test)
except:
    raise NotImplementedError()

## Step 4.3 Construct, Train, Evaluate, Results in TensorFlow

### Step 4.3.1

Construct a DNNClassifier with two hidden layers of 5 units each and store it as `dnn_55`. For reference, [here](https://www.tensorflow.org/get_started/premade_estimators) is an example of using a DNNClassifier.

In [57]:
# TODO: Create DNNClassifier
tf.set_random_seed(42)
import tempfile
# YOUR CODE HERE
try:
    directory = tempfile.mkdtemp()
    dnn_55 = tf.estimator.DNNClassifier(model_dir = directory, feature_columns = label_col, hidden_units = [5,5])

except:
    raise NotImplementedError()

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_id': 0, '_save_summary_steps': 100, '_service': None, '_task_type': 'worker', '_log_step_count_steps': 100, '_num_ps_replicas': 0, '_keep_checkpoint_every_n_hours': 10000, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fb14f987a58>, '_global_id_in_cluster': 0, '_master': '', '_eval_distribute': None, '_tf_random_seed': None, '_is_chief': True, '_device_fn': None, '_protocol': None, '_save_checkpoints_secs': 600, '_num_worker_replicas': 1, '_keep_checkpoint_max': 5, '_evaluation_master': '', '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_model_dir': '/tmp/tmpar9pjku4', '_train_distribute': None, '_save_checkpoints_steps': None, '_experimental_distribute': None}


### Step 4.3.2

Construct a LinearClassifier here and store it as `tf_lin`.

In [58]:
# TODO: Construct your LinearClassifier here.
tf.set_random_seed(42)
# YOUR CODE HERE
try:
    directory = tempfile.mkdtemp()
    tf_lin = tf.estimator.LinearClassifier(model_dir = directory, feature_columns = label_col)
except:
    raise NotImplementedError()

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_id': 0, '_save_summary_steps': 100, '_service': None, '_task_type': 'worker', '_log_step_count_steps': 100, '_num_ps_replicas': 0, '_keep_checkpoint_every_n_hours': 10000, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fb14f987550>, '_global_id_in_cluster': 0, '_master': '', '_eval_distribute': None, '_tf_random_seed': None, '_is_chief': True, '_device_fn': None, '_protocol': None, '_save_checkpoints_secs': 600, '_num_worker_replicas': 1, '_keep_checkpoint_max': 5, '_evaluation_master': '', '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_model_dir': '/tmp/tmp9cwwewxe', '_train_distribute': None, '_save_checkpoints_steps': None, '_experimental_distribute': None}


### Step 4.3.3

Write a function `train_evaluate(m, num_steps)` that takes the model and a number of steps as arguments, trains the model over the training data, evaluates on the test data, sorts the results of the evaluate operation by key, prints the keys and their values, and returns the accuracy. 

In [63]:
# TODO: Write your train_evaluate(m, num_steps) function here.
# YOUR CODE HERE
try:
    def train_evaluate(m, num_steps):
        m.train(input_fn = train_input_fn, steps = num_steps)
        eval_result = m.evaluate(input_fn = test_input_fn, steps = 1)
        for x in sorted(eval_result):
            print("%s: %s" % (x, eval_result[x]))
        return eval_result["accuracy"]
except:
    raise NotImplementedError()

In [64]:
# Results, as a list of dictionaries
classifier_results = []

classifier_results.append({'Classifier': 'DNN', 'Param': 'Hidden=(5,5)', 'Score': train_evaluate(dnn_55, 1000)})
classifier_results.append({'Classifier': 'Lin', 'Param': '',             'Score': train_evaluate(tf_lin, 1000)})

display(pd.DataFrame(classifier_results))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpar9pjku4/model.ckpt-2000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 2000 into /tmp/tmpar9pjku4/model.ckpt.
INFO:tensorflow:loss = 232.91771, step = 2001
INFO:tensorflow:global_step/sec: 217.936
INFO:tensorflow:loss = 229.50418, step = 2101 (0.460 sec)
INFO:tensorflow:global_step/sec: 447.678
INFO:tensorflow:loss = 226.29303, step = 2201 (0.224 sec)
INFO:tensorflow:global_step/sec: 452.088
INFO:tensorflow:loss = 223.45604, step = 2301 (0.221 sec)
INFO:tensorflow:global_step/sec: 483.119
INFO:tensorflow:loss = 220.79092, step = 2401 (0.208 sec)
INFO:tensorflow:global_step/sec: 487.54
INFO:tensorflow:loss = 218.33636, step = 2501 (0.203 sec)
INFO:tensorflow:global_step/sec: 485.867
INFO:tensorflow:loss = 216.12

Classifier         Param     Score
0        DNN  Hidden=(5,5)  0.975785
1        Lin                0.974888

In [65]:
# Don't delete this cell.